# Board orientation classification

# Import the library we need to process data, build and train a ML model

In [1]:
import pandas as pd
import numpy as np #import numpy library, used for arithmetic
from sklearn.model_selection import train_test_split #mpodule that allow us to split our train and test sets
from sklearn.svm import SVC #the machine learning classifier model

# Data preparation

## Data loading

The data have been captured with my software [Datasets Builder](https://github.com/BaptisteZloch/MakeIT-How-to-build-your-own-datasets-with-Datasets-Builder). This software have been presented in my previous [video](https://youtu.be/SJ46CEvt-nY).

In [2]:
df = pd.read_csv('/content/Orientation.csv',sep=",")  #read the csv file
df.head() #show the 5 first rows of the dataframe

,X,Y,Z,Orientation
0,0.94,8.90,-1.50,DOWN
1,1.21,9.59,-1.07,DOWN
2,1.09,9.14,-2.36,DOWN
3,1.03,9.12,-4.16,DOWN
4,1.38,9.12,-6.34,DOWN


In [3]:
df['Orientation'].value_counts() #this method inspect the count for each value in a column

HORIZONTAL    988
UP            727
DOWN          303
Name: Orientation, dtype: int64

## Data wrangling

In [4]:
#This function will map the corresponding UP, DOWN, HORIZONTAL keyword to numerical values
def orientationToNumber(x):
  if x=='UP':
    return 1
  elif x=='DOWN':
    return 2
  else:
    return 3

df['Orientation'] = df['Orientation'].apply(orientationToNumber) #apply command will apply a function to a whole vector
df.head() #let's see the result

,X,Y,Z,Orientation
0,0.94,8.90,-1.50,2
1,1.21,9.59,-1.07,2
2,1.09,9.14,-2.36,2
3,1.03,9.12,-4.16,2
4,1.38,9.12,-6.34,2


In [5]:
labels = np.array(df.pop('Orientation')) #extract the label column and convert it into numpy ndarray
features = np.array(df) #convert our dataframe into numpy ndarray

In [6]:
labels

array([2, 2, 2, ..., 3, 3, 3])

In [7]:
features

array([[ 0.94,  8.9 , -1.5 ],
       [ 1.21,  9.59, -1.07],
       [ 1.09,  9.14, -2.36],
       ...,
       [10.49, -0.19, -1.24],
       [10.38, -0.11, -1.13],
       [10.27, -0.12, -0.95]])

## Data spliting

In [8]:
#Split the dataset into training set 85% and test set 15%
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.15,shuffle=True) 

# Model building

## Model creating

In [9]:
model = SVC(gamma=0.001) #lets instanciate our model

## Model training

In [10]:
model.fit(train_features, train_labels) #we train our model

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## Model evalutating

In [11]:
print("Accuracy for the model : {} %".format(100*round(model.score(test_features,test_labels),2))) #we verify the accuracy of our model

Accuracy for the model : 100.0 %


# Convertion of the model

## Installing the convertion library

In [12]:
!pip install micromlgen

  Created wheel for micromlgen: filename=micromlgen-1.1.23-py3-none-any.whl size=29096 sha256=e7e75759f14b6e2c888e7a3fc5c8e39fb2e0c031842ea663bbbb831af435fd2e
  Stored in directory: /root/.cache/pip/wheels/03/c3/bc/b81eaa66a4664cd550594388499cd8140ba1a13c5986a9c407
Successfully built micromlgen


## Import the library

In [13]:
from micromlgen import port

## Create a class mapping between labels and classes

In [14]:
LABELS = ['UP','DOWN','HORIZONTAL'] #our existing labels
classMap = {} #create an empty dict
for i, label in zip(range(3),LABELS): #interate over the range and the labels at the same time 
  classMap[i]=label #fill our dict

print(classMap)

{0: 'UP', 1: 'DOWN', 2: 'HORIZONTAL'}


## Convert the model

In [15]:
c_code = port(model,classmap=classMap) #convert our model

#Let's write it into a .h file
modelFile = open("model.h", "w")
modelFile.write(c_code)
modelFile.close()

#Let's print the size of the .h file
import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

Header file, model.h, is 12,062 bytes.

Open the side panel (refresh if needed). Double click model.h to download the file.
